Fine tune Deepseek R1 Distilled Qwen 1.5B model w/ LoRA

# Necessary Imports

In [ ]:
%pip install datasets trl peft accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Load Dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations") # Small number of rows
dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 2260
    })
    test: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 119
    })
})

# Get Base Model and Tokenizer

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import SFTConfig, SFTTrainer, setup_chat_format # Supervised fine-tune
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load model
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

model = AutoModelForCausalLM.from_pretrained(
    model_name
).to(device)

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
)

# Finetune model name
finetuned_model_name = "DeepSeek-R1-Distill-Qwen-1.5B-finetuned-smoltalk-everyday-conversations"

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


# Set Hyperparameters

## LoRA Config

In [ ]:
from peft import LoraConfig

# LoRA config
rank_dimension = 6 # Decrease to speed up training
lora_alpha = 2 * rank_dimension
lora_dropout = 0.05 # Helps w/ over-fitting

peft_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=rank_dimension, # Usually 4-32
    lora_alpha=lora_alpha, # Usually 2x rank
    lora_dropout=lora_dropout,
    bias="none", # Biases to be updated during training
    target_modules="all-linear" # Apply LoRA to all linear embeddings
)

## Training arguments

In [ ]:
training_args = SFTConfig(
    output_dir=finetuned_model_name,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True, # Saves memory but is more computationally expensive
    optim="adamw_torch_fused", # More efficiency
    logging_steps=10,
    max_grad_norm=0.3, # Gradient clipping threshold
    warmup_ratio=0.03, # Amount of dataset to increase lr to specified amount
    lr_scheduler_type="constant", # No change in lr after warmup
    learning_rate=2e-4,
    save_strategy="epoch", # Save checkpoint every epoch
    bf16=True, # Mixed precision to speed up training
    report_to="none", # Disable WandB logging
)

# Train Model

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config, # LoRA config
)

Converting train dataset to ChatML:   0%|          | 0/2260 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/2260 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2260 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2260 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/119 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/119 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/119 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/119 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,3.073300
20,2.273300
30,1.986800
40,1.959300
50,1.730500
60,1.811900
70,1.779600
80,1.783400
90,1.736600
100,1.667700


TrainOutput(global_step=565, training_loss=1.6710539699655718, metrics={'train_runtime': 2172.9719, 'train_samples_per_second': 1.04, 'train_steps_per_second': 0.26, 'total_flos': 3848548529147904.0, 'train_loss': 1.6710539699655718})

# Push Model to Hugging Face Hub

In [ ]:
from peft import AutoPeftModelForCausalLM

# Load PEFT model on CPU

model = AutoPeftModelForCausalLM.from_pretrained(
    "/content/DeepSeek-R1-Distill-Qwen-1.5B-finetuned-smoltalk-everyday-conversations/checkpoint-565", # Load just-trained model
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16, # Make it faster
)

merged_model = model.merge_and_unload()

In [ ]:
from huggingface_hub import notebook_login

notebook_login() # Login to account

In [ ]:
merged_model.push_to_hub(finetuned_model_name) # Push to hub
tokenizer.push_to_hub(finetuned_model_name)

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/avanishd/DeepSeek-R1-Distill-Qwen-1.5B-finetuned-smoltalk-everyday-conversations/commit/b363ebd0e8bfb7fb43e6140b698088b706628b49', commit_message='Upload Qwen2ForCausalLM', commit_description='', oid='b363ebd0e8bfb7fb43e6140b698088b706628b49', pr_url=None, repo_url=RepoUrl('https://huggingface.co/avanishd/DeepSeek-R1-Distill-Qwen-1.5B-finetuned-smoltalk-everyday-conversations', endpoint='https://huggingface.co', repo_type='model', repo_id='avanishd/DeepSeek-R1-Distill-Qwen-1.5B-finetuned-smoltalk-everyday-conversations'), pr_revision=None, pr_num=None)